In [1]:
import numpy as np
import pandas as pd
import os

import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pyampute
import pickle 
import time
import ast 

from scipy.stats import mstats
from scipy.stats.mstats import winsorize
from scipy import stats
from xgboost import XGBRegressor
from sklearn import tree
from pyampute.ampute import MultivariateAmputation
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from lifelines import CoxPHFitter, WeibullFitter, WeibullAFTFitter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tableone import TableOne 
from os.path import isfile, join
from sklearn.metrics import mean_absolute_error, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from statsmodels.gam.tests.test_penalized import df_autos
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import norm
from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from app_transition_dict import get_transition_dict, get_transition_code
import warnings 
warnings.filterwarnings('ignore')

drive = 'G'
main_path = drive + ':/Shared drives/CKD_Progression/data/CKD_COHORT_Jan2010_Mar2024_v3.csv'
data_path = drive + ':/Shared drives/CKD_Progression/data/'
docs_path = drive + ':/Shared drives/CKD_Progression/docs/'
save_path = drive + ':/Shared drives/CKD_Progression/save/'
resu_path = drive + ':/Shared drives/CKD_Progression/result/'
covariates_path = docs_path + 'covariates.csv'
removecols_path = docs_path + 'remove_columns.csv'

In [2]:
def adjust_dates(row, date_columns):
    last_date = pd.to_datetime('2010-01-01') 
    max_date  = pd.to_datetime('2023-12-31')
    for col in date_columns:
        if pd.notna(row[col]):
            if row[col] <= last_date:
                row[col] = last_date + timedelta(days = 1)
            last_date = row[col]
            if last_date > max_date:
                last_date = max_date
                row[col] = max_date
    return row

def calculate_duration(row):
    from_date = row[f"{row['fr']}_date"]
    to_date   = row[f"{row['to']}_date"]
    duration   = np.abs((to_date.year - from_date.year) * 12 + (to_date.month - from_date.month))
    return duration

def calculate_duration(row):
    from_date = row[f"{row['fr']}_date"]
    to_date   = row[f"{row['to']}_date"]
    duration_in_days = np.abs((to_date - from_date).days)
    duration_in_months = duration_in_days / 30
    return np.abs(duration_in_months)

def get_covariates(_path = 'G:/Shared drives/CKD_MS/multi_stage_CKD/data/covariates_080823.xlsx'):
    covariates_df = pd.read_excel(_path)
    covariates = covariates_df[covariates_df['ignore'] != 'IGNORE']
    covariates = covariates['variable'].tolist()
    covariates_ord = covariates_df[covariates_df['variable_type'] == 'ordinal']    ['variable'].tolist()
    covariates_con = covariates_df[covariates_df['variable_type'] == 'continuous'] ['variable'].tolist()
    covariates_dic = covariates_df[covariates_df['variable_type'] == 'dichotomous']['variable'].tolist()
    return covariates, covariates_ord, covariates_con, covariates_dic
    
def univariate_df(patients, covariates, path = save_path + 'qoc_cohort_ver002.csv'):
    df = pd.read_csv(path)
    df['ANTI_PL']     = df[['asa', 'other_antipl', 'p2y12', 'pde']].max(axis = 1)
    df['PHOS_BINDER'] = df[['phos_binder_alu', 'phos_binder_ca', 'phos_binder_lanthanum', 'phos_binder_sevelamer']].max(axis = 1)
    df = df.fillna(1)
    df = df[covariates]
    df = df[df['ENC_HN'].isin(patients)]
    return df

def fill_dates(row):
    fr_col = row['fr'] + '_date'
    to_col = row['to'] + '_date'
    
    row['start_date'] = row[fr_col] if fr_col in df.columns else None
    row['stop_date']  = row[to_col] if to_col in df.columns else None
    return row

def calculate_tstart(row):
    if row['fr'] == 'CKD3A':
        return (((row['begin'] - row['diag_date']).days)/365) * 12
    else:
        return (((row['end'] - row['begin']).days)/365) * 12
    
def get_second_to_last(transition):
    transition_tuple = eval(transition)
    if len(transition_tuple) > 1:
        return transition_tuple[-2]
    else:
        return None

In [3]:
covariates, covariates_ord, covariates_con, covariates_dic = get_covariates()
covariates_con.remove('Renal_eGFR')
covariates_dic.remove('dead')
covariates_dic.remove('CVD')
covariates_dic.remove('T2DM')

multistage = pd.read_csv(save_path + 'multistage_ver010.csv')
patients   = multistage['ENC_HN'].tolist()
assert len(patients) == 23693 

df_con = pd.read_csv(resu_path + 'imputation/decision_tree_early_stop/IMPUTED_DT.csv')
df_con['ENC_HN'] = patients
df_con['BMI'] = np.divide(df_con['BW'], np.divide(df_con['HIGH'], 100) ** 2)
df_con = df_con.drop(columns = ['CVD'])
df_dic = univariate_df(patients, ['ENC_HN'] + covariates_dic)
assert df_con.shape[0] == 23693
assert df_dic.shape[0] == 23693

df_main = pd.merge(df_con, df_dic, on = 'ENC_HN', how = 'inner', indicator = True)
df_main = df_main.drop(columns = ['_merge'])
assert df_main.shape[0] == 23693

covariates = df_main.columns.tolist()
covariates.remove('ENC_HN')

In [ ]:
transition_list = list(get_transition_dict().keys())
transitions = [
    'CKD3A_CKD3B', 'CKD3A_CKD4', 'CKD3A_CKD5A', 'CKD3A_CKD5B', 'CKD3A_CVD', 'CKD3A_DEATH',
    'CKD3B_CKD4',  'CKD3B_CKD5A', 'CKD3B_CKD5B', 'CKD3B_CVD', 'CKD3B_DEATH',
    'CKD4_CKD5A',  'CKD4_CKD5B', 'CKD4_CVD', 'CKD4_DEATH',
    'CKD5A_CKD5B', 'CKD5A_CVD', 'CKD5A_DEATH',
    'CKD5B_CVD',   'CKD5B_DEATH',
    'CVD_CKD3B',   'CVD_CKD4', 'CVD_CKD5A', 'CVD_CKD5B', 'CVD_DEATH']

transitions_dict = {transition: index + 1 for index, transition in enumerate(transitions)}
date_columns = ['CKD3A_date',   'CKD3B_date',   'CKD4_date',    'CKD5A_date',   'CKD5B_date',   'CVD_date', 'DEAD_date']
stat_columns = ['CKD3A_status', 'CKD3B_status', 'CKD04_status', 'CKD5A_status', 'CKD5B_status', 'CVD00_status', 'DEATH_status']


df = pd.read_csv(save_path + 'multistage_ver010.csv')
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df = df[['ENC_HN', 'transition'] + date_columns + transitions]
df = pd.merge(df, df_main, on = 'ENC_HN', how = 'inner', indicator = True)
df = df[df['_merge'] == 'both']
df = df.drop(columns = ['_merge'])
df = df[['ENC_HN', 'transition'] + covariates + date_columns + transitions]

assert df.shape[0] == 23693

change_columns = df.columns[df.columns.get_loc('CKD3A_CKD3B'):].tolist()
for column in change_columns:
    df.rename(columns = {column: 'status' + column}, inplace = True)

In [93]:
long_df = pd.wide_to_long(df, stubnames = ['status'], i = 'ENC_HN', j = 'variables', suffix = '\w+').reset_index()
long_df['transit'] = long_df['variables'].map(transitions_dict)
long_df = long_df.sort_values(['ENC_HN', 'transit'], ascending = [True, True]).reset_index(drop = True)
long_df['fr'] = long_df['variables'].apply(lambda x: x.split('_')[0].upper())
long_df['to'] = long_df['variables'].apply(lambda x: x.split('_')[1].upper())
long_df[date_columns] = long_df[date_columns].apply(pd.to_datetime, format = '%m/%d/%Y')
long_df['to'] = long_df['to'].replace('DEATH', 'DEAD')
long_df['duration'] = long_df.apply(calculate_duration, axis = 1)
long_df = long_df[['ENC_HN', 'transition', 'fr', 'to', 'status', 'duration'] + date_columns + covariates]

In [94]:
long_df = long_df[long_df.apply(lambda x: x['fr'] in x['transition'], axis = 1)]
long_df = long_df.apply(fill_dates, axis = 1)

# date_columns.remove('CKD4_date')
# date_columns.remove('CKD5A_date')
# date_columns.remove('CKD5B_date')
# date_columns.remove('CVD_date')

execute = False
if execute:
    long_df = long_df.drop(columns = covariates)   ## TODEL
    long_df = long_df.drop(columns = date_columns) ## TODEL

In [38]:
long_df.to_csv(save_path + 'long_mock2.csv', index = False)

In [95]:
## START HERE FOR SIMULATIONS

# long_df = pd.read_csv(save_path + 'long_mock2.csv') ## TODEL
long_df['start_date'] =  pd.to_datetime(long_df['start_date'])
long_df['stop_date']  =  pd.to_datetime(long_df['stop_date']) 
long_df['diag_date'] = long_df.groupby(['ENC_HN'])['start_date'].transform('min')
long_df['diag_date'] = pd.to_datetime(long_df['diag_date'], errors='coerce')
long_df['unique_tstop'] = long_df['ENC_HN'] + ' ' + long_df['fr']
long_df['begin'] = long_df.groupby(['unique_tstop'])['start_date'].transform('min')
long_df['end']   = long_df.groupby(['unique_tstop'])['stop_date'] .transform('min')
long_df['tstart'] = long_df.apply(calculate_tstart, axis = 1)
long_df['tstops'] = (((long_df['end'] - long_df['diag_date']).dt.days)/365) * 12

mapping = {}
for idx, row in long_df.iterrows():
    enc_hn = row['ENC_HN']
    current_fr = row['fr']
    next_fr = row['to']
    if enc_hn not in mapping:
        mapping[enc_hn] = {}
    # Replace the tstart values for 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B', and 'CVD'
    if next_fr in ['CKD3B', 'CKD4', 'CKD5A', 'CKD5B', 'CVD']:
        # Map the current 'fr' to its 'tstops'
        mapping[enc_hn][next_fr] = row['tstops']

# Update tstart values in long_df
for idx, row in long_df.iterrows():
    enc_hn = row['ENC_HN']
    current_fr = row['fr']
    # If this 'fr' has a mapped tstop from a previous state, update its tstart
    if current_fr in mapping[enc_hn]:
        long_df.at[idx, 'tstart'] = mapping[enc_hn][current_fr]
long_df['time'] = long_df['tstops'] - long_df['tstart']
long_df['begin'] = long_df.groupby(['unique_tstop'])['start_date'].transform('min')
long_df['end']   = long_df.groupby(['unique_tstop'])['stop_date'] .transform('min')
long_df = long_df.drop(columns = ['unique_tstop', 'start_date', 'stop_date', 'duration'])
long_df['begin'], long_df['end'] = long_df['begin'].dt.date, long_df['end'].dt.date
long_df = long_df[['ENC_HN', 'diag_date', 'transition', 'fr', 'to', 'status', 'begin', 'end', 'tstart', 'tstops', 'time'] + date_columns + covariates] 

In [96]:
negative_transitions = pd.read_excel(docs_path + 'negative_transitions.xlsx')
negative_transitions = negative_transitions[negative_transitions['death'] == 2]
negative_transitions = negative_transitions['transitions'].tolist()

mask_transition = (long_df['transition'].isin(negative_transitions)) & (long_df['fr'] == 'CVD')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

In [97]:
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD')") & (long_df['fr'] == 'CKD4')
tstops_values = long_df[(long_df['fr'] == 'CKD3B') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']


mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD')") & (long_df['fr'] == 'CKD5A')
tstops_values = long_df[(long_df['fr'] == 'CKD4') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

In [98]:
## DEATH
negative_transitions = pd.read_excel(docs_path + 'negative_transitions.xlsx')
negative_transitions = negative_transitions[negative_transitions['death'] == 1]
negative_transitions = negative_transitions['transitions'].tolist()

long_df['second'] = long_df['transition'].apply(get_second_to_last)
long_df['second'] = [second if fr == second else 0 for fr, second in zip(long_df['fr'], long_df['second'])]
mask_transition = (long_df['transition'].isin(negative_transitions)) & (long_df['second'] != 0) 
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'DEAD_date']
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']
long_df = long_df.drop(columns = ['second'])

In [99]:
# CVD interchange
negative_transitions = pd.read_excel(docs_path + 'negative_transitions.xlsx')
negative_transitions = negative_transitions[negative_transitions['death'] == 0]
negative_transitions = negative_transitions['transitions'].tolist()
mask_transition = (long_df['transition'].isin(negative_transitions))

# ('CKD3A', 'CKD3B', 'CVD', 'CKD4') 
long_df['second'] = long_df['transition'].apply(get_second_to_last)
long_df['second'] = [second if fr == second else 0 for fr, second in zip(long_df['fr'], long_df['second'])]
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4')") & (long_df['second'] != 0) 
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'CKD4_date']
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']
long_df = long_df.drop(columns = ['second'])

# ('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CVD')
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'CKD4_date']
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'CKD5A_date']
tstops_values = long_df[(long_df['fr'] == 'CVD') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD5A')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD5A')
tstops_values = long_df[(long_df['fr'] == 'CKD4') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']


# ('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CVD')
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'CKD4_date']
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end']    = long_df.loc[mask_transition, 'CKD5A_date']
tstops_values = long_df[(long_df['fr'] == 'CVD') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5A')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD5B_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']
mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5A')
tstops_values = long_df[(long_df['fr'] == 'CKD4') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5B')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5B')
tstops_values = long_df[(long_df['fr'] == 'CKD5A') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

# ('CKD3A', 'CVD', 'CKD3B')
mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B')") & (long_df['fr'] == 'CKD3B')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']


# ('CKD3A', 'CVD', 'CKD3B', 'CKD4')
mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4')") & (long_df['fr'] == 'CKD3B')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD4_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4')") & (long_df['fr'] == 'CKD4')
tstops_values = long_df[(long_df['fr'] == 'CKD3B') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']


# ('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')
mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD3B')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD4_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD5A_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD5A')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')") & (long_df['fr'] == 'CKD5A')
tstops_values = long_df[(long_df['fr'] == 'CKD4') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

# ('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')
mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD3B')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD4_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD4')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD5A_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD4')
tstops_values = long_df[(long_df['fr'] == 'CKD3B') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5A')
long_df.loc[mask_transition, 'end'] = long_df.loc[mask_transition, 'CKD5B_date']
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5A')
tstops_values = long_df[(long_df['fr'] == 'CKD4') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5B')
long_df.loc[mask_transition, 'end'] = pd.to_datetime('12-31-2023')
long_df['end'] = pd.to_datetime(long_df['end'])
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days)/365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']

mask_transition = (long_df['transition'] == "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')") & (long_df['fr'] == 'CKD5B')
tstops_values = long_df[(long_df['fr'] == 'CKD5A') & (long_df['to'] == 'DEAD')][['ENC_HN', 'tstops']]
tstops_values = tstops_values.set_index('ENC_HN')['tstops'].to_dict()
long_df.loc[mask_transition, 'tstart'] = long_df.loc[mask_transition, 'ENC_HN'].map(tstops_values)
long_df.loc[mask_transition, 'tstops'] = (((long_df.loc[mask_transition, 'end'] - long_df.loc[mask_transition, 'diag_date']).dt.days) / 365) * 12
long_df.loc[mask_transition, 'time']   = long_df.loc[mask_transition, 'tstops'] - long_df.loc[mask_transition, 'tstart']


In [100]:
transitions = [
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'CKD5B')",
    "('CKD3A', 'CVD', 'CKD3B')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'DEATH')",
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A')",
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'CKD5B')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')",
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B')",
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'DEATH')",
    "('CKD3A', 'CVD', 'CKD3B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CVD', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B', 'DEATH')"
]


for transition in transitions:
    if 'CVD' in transition:
        transition_tuple = eval(transition)
        cvd_index = transition_tuple.index('CVD')
        states_after_cvd = transition_tuple[cvd_index + 1:]
        states_after_cvd = tuple(states_after_cvd)
        mask = (
            (long_df['transition'] == transition) &
            (long_df['fr'].isin(states_after_cvd)) &
            (long_df['to'] == 'CVD')
        )
        long_df = long_df[~mask]

long_df = long_df.reset_index(drop = True)

In [101]:
transitions = [
    "('CKD3A', 'CKD3B', 'CVD')",
    "('CKD3A', 'CKD3B', 'CVD', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A')",
    "('CKD3A', 'CKD3B', 'CVD', 'CKD4', 'CKD5A', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'CKD5B', 'DEATH')",

    "('CKD3A', 'CKD3B', 'CKD4', 'CVD')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'CKD5B')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CVD', 'CKD5A', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'CKD5B')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CVD', 'CKD5B', 'DEATH')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B', 'CVD')",
    "('CKD3A', 'CKD3B', 'CKD4', 'CKD5A', 'CKD5B', 'CVD', 'DEATH')"]

long_df['transition_parsed'] = long_df['transition'].apply(ast.literal_eval)
transitions_parsed = [ast.literal_eval(t) for t in transitions]
long_df['valid_transition'] = long_df['transition_parsed'].apply(lambda x: x in transitions_parsed)

def should_keep_row(row):
    if not row['valid_transition']:
        return True
    pathway = row['transition_parsed']
    fr_state = row['fr']
    to_state = row['to']
    try:
        cvd_index = pathway.index('CVD')
    except ValueError:
        return True
    states_before_cvd = [state for state in pathway[:cvd_index] if state != 'CKD3A']
    if fr_state == 'CVD' and to_state in states_before_cvd:
        return False
    else:
        return True
long_df['keep_row'] = long_df.apply(should_keep_row, axis = 1)
long_df = long_df[long_df['keep_row']].drop(columns = ['transition_parsed', 'valid_transition', 'keep_row'])

In [102]:
exclude = long_df[long_df['time'] < 0]['ENC_HN'].unique().tolist()
long_df = long_df[~long_df['ENC_HN'].isin(exclude)]
long_df.to_csv(save_path + 'multi_state_long_ver013.csv', index = False)

In [103]:
long_df.shape

(313871, 47)

In [88]:
# TODEL
df = pd.read_csv(save_path + 'multi_state_long_ver013.csv')
unique_transitions_per_patient = df.groupby('ENC_HN')['transition'].first().reset_index()
patient_counts_by_transition = unique_transitions_per_patient['transition'].value_counts().reset_index()
patient_counts_by_transition.columns = ['transition', 'patient_count']
patient_counts_by_transition.to_csv(save_path + 'msm_check_02.csv', index = False)